### 이미지 로데이터 파일에서 데이터 추출


In [9]:
## [1] 파일 입출력 방식
## with open() as ~

file_path = "../data/MNIST/t10k-images-idx3-ubyte/t10k-images.idx3-ubyte"


In [10]:
# 파일 읽을 때 고려사항 => mode  = rb : 바이트 읽기 모드
with open(file_path, "rb") as f:
    all_data = f.read()


In [11]:
print(type(all_data), len(all_data))
print(all_data[:100])


<class 'bytes'> 7840016
b"\x00\x00\x08\x03\x00\x00'\x10\x00\x00\x00\x1c\x00\x00\x00\x1c\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00"


In [13]:
# 파일 읽을 때 고려사항 => mode  = rb : 바이트 읽기 모드
with open(file_path, "rb") as f:
    magic_code = f.read(4)
    print(magic_code)

    data_length = f.read(4)
    print(data_length)

    row = f.read(4)
    print(row)

    col = f.read(4)
    print(col)


b'\x00\x00\x08\x03'
b"\x00\x00'\x10"
b'\x00\x00\x00\x1c'
b'\x00\x00\x00\x1c'


-   [2] 파이썬의 byte 데이터 변환 모듈 struct 활용


In [25]:
### ===> 모듈 로딩
import struct


In [26]:
### ===> 데이터 추출 및 저장 관련 변수
save_file = "../data/MNIST/mnist.csv"
